依赖项 | 版本--- | ---
SatPy | 0.25.1PyResample | 1.17.0PySpectral | 0.10.4Trollimage | 1.14.0PyKdtree | 1.3.4XArray | 0.16.1Dask | 2.30.0ImageIO | 2.9.0FFmpeg | 4.3.1

# GOES-16 ABI - 真彩色动画演示 - 飓风佛罗伦萨


此示例展示了如何将多个时间步长的ABI数据合并为一个mp4电影文件，作为动画呈现。该示例使用了2018年9月11日从1300Z到1440Z期间GOES-16 ABI仪器获取的100帧中尺度图像，每分钟一帧。在Jupyter Notebook中使用"%%time"魔法命令来计时整个单元格的执行时间。此示例在使用8个Dask线程工作者的2015款MacBook上耗时约8分30秒。
制作的视频已上传至 Twitter：https://twitter.com/PyTrollOrg/status/1039555399433834497
全分辨率视频：https://www.ssec.wisc.edu/~davidh/satpy/true_color_20180911_130021.mp4
电影中的帧是已应用太阳天顶角和瑞利散射校正的500米分辨率真彩色图像。还使用了比例锐化技术，以从较低分辨率的输入波段中提取更多细节。所有这些在satpy的ABI 'true_color'合成中均已预配置，并通过Python实现。

In [ ]:
%%time
import os
import warnings
import logging
from satpy import Scene
from glob import glob
from satpy import MultiScene
logging.basicConfig(level=logging.ERROR)  # don't show warnings
warnings.simplefilter('ignore')  # don't show warnings

BASE_DIR = '/data/satellite/abi/20180911_florence_100'

mscn = MultiScene.from_files(glob(os.path.join(BASE_DIR, 'OR*-RadM1*.nc')), reader='abi_l1b')
mscn.load(['true_color'])
new_mscn = mscn.resample(resampler='native')
new_mscn.save_animation('{name}_{start_time:%Y%m%d_%H%M%S}.mp4', fps=5)

In [ ]:
from IPython.display import Video
Video('true_color_20180911_130021.mp4', width=480, height=480)

# 高级用法
也可以在多进程环境中使用 dask 一次计算多个动画帧。我们首先通过创建一个 dask 的 `Client` 对象来实现这一点，该对象会为我们创建外部的 dask 工作节点。然后，我们将这个 client 对象和一个 `batch_size` 一起传递给 `save_animation`。这会指示 Satpy 使用 dask 创建的并行多进程工作节点一次计算 `16` 帧。如果没有这个设置，Satpy 默认会一次只计算一帧。根据您的系统，这可以节省数分钟的处理时间。
注意：下面创建的 `Client` 包括提供一个仪表板，用于查看 dask 正在处理的任务。

In [ ]:
from dask.distributed import Client
client = Client()
client

In [ ]:
%%time
import os
import warnings
import logging
from satpy import Scene
from glob import glob
from satpy import MultiScene
logging.basicConfig(level=logging.ERROR)  # don't show warnings
warnings.simplefilter('ignore')  # don't show warnings

BASE_DIR = '/data/satellite/abi/20180911_florence_100'

mscn = MultiScene.from_files(glob(os.path.join(BASE_DIR, 'OR*-RadM1*.nc')), reader='abi_l1b')
mscn.load(['true_color'])
new_mscn = mscn.resample(resampler='native')
new_mscn.save_animation('{name}_{start_time:%Y%m%d_%H%M%S}.mp4', fps=5, batch_size=16, client=client)